## 200. Number of Islands
- Description:
  <blockquote>
    Given an m x n 2D binary grid grid which represents a map of '1's (land) and '0's (water), return the number of islands.

    An island is surrounded by water and is formed by connecting adjacent lands horizontally or vertically. You may assume all four edges of the grid are all surrounded by water.

    

    Example 1:

    Input: grid = [
      ["1","1","1","1","0"],
      ["1","1","0","1","0"],
      ["1","1","0","0","0"],
      ["0","0","0","0","0"]
    ]
    Output: 1

    Example 2:

    Input: grid = [
      ["1","1","0","0","0"],
      ["1","1","0","0","0"],
      ["0","0","1","0","0"],
      ["0","0","0","1","1"]
    ]
    Output: 3

    

    Constraints:

        m == grid.length
        n == grid[i].length
        1 <= m, n <= 300
        grid[i][j] is '0' or '1'.
  </blockquote>

- URL: [Problem_URL](https://leetcode.com/problems/number-of-islands/description/)

- Topics: Graph, DFS, BFS, Union Find

- Difficulty: Medium

- Resources: example_resource_URL

### Solution 1, Recursive DFS, marking visited cells by changing them to '#'

R - rows, C- cols

- Time Complexity: O(R * C)
- Space Complexity: O(R * C)
  -  in case that the grid map is filled with lands where DFS goes by R×C deep.

In [ ]:
class Solution:
    def numIslands(self, grid: List[List[str]]) -> int:
        if not grid:
            return 0

        def dfs(rw, cl):
            # Check row and col are within grid bounds
            if not (0 <= rw < len(grid)) or not (0 <= cl < len(grid[0])):
                return
            
            # Check if cell contains land and is unvisited
            if grid[rw][cl] != "1":
                return
            
            # Mark cell as visited by setting the cells value to '#'
            grid[rw][cl] = '#'

            # Visit adjacent cells vertically and horizontally
            directions = ((rw-1, cl), (rw+1, cl), (rw, cl-1), (rw, cl+1))

            for rw, cl in directions:
                dfs(rw, cl)


        
        island_count = 0
        for rw in range(len(grid)):
            for cl in range(len(grid[0])):
                # If cell contains land, visit neighboring cells (horizontally or vertically) in dfs manner
                # to find connected land cells that make up a single island
                if grid[rw][cl] == "1":
                    dfs(rw, cl)
                    island_count += 1
        
        return island_count

### Solution 2, Most Optimum, BFS by marking visited cells by changing them to '#'
R - rows, C- cols

- Time Complexity: O(R * C)
- Space Complexity: O(min(R,C))
  - Because in worst case where the grid is filled with lands, the size of queue can grow up to min(M,N)
  - [visual explanation](https://imgur.com/gallery/bfs-2d-grid-of-m-x-n-M58OKvB)

In [ ]:
from collections import deque


class Solution:
    def numIslands(self, grid: List[List[str]]) -> int:
            if not grid:
                return 0

            def bfs(rw, cl):
                queue = deque()
                queue.append((rw, cl))

                # Mark cell as visited by setting the cells value to '#'
                grid[rw][cl] = '#'

                while queue:
                    rw, cl = queue.popleft()

                    directions = ((rw-1, cl), (rw+1, cl), (rw, cl-1), (rw, cl+1))
                    
                    for rd, cd in directions:
                        # If row and col are within grid bounds and current cell is a land cell, Visit adjacent cells vertically and horizontally
                        if (0 <= rd < len(grid)) and (0 <= cd < len(grid[0])) and grid[rd][cd] == '1':
                            queue.append((rd, cd))
                            grid[rd][cd] = '#'


            
            island_count = 0
            for rw in range(len(grid)):
                for cl in range(len(grid[0])):
                    # If cell contains land, visit neighboring cells (horizontally or vertically) in bfs manner
                    # to find connected land cells that make up a single island
                    if grid[rw][cl] == "1":
                        bfs(rw, cl)
                        island_count += 1
            
            return island_count

### Solution 3, Union Find AKA Disjoint Set
R - rows, C- cols

- Time Complexity: O(R * C)
  - Union operation takes essentially constant time when UnionFind is implemented with both path compression and union by rank.
- Space Complexity: O(R * C)
  - as required by UnionFind data structure

In [ ]:
# My Union Find Solution
class Solution:
    def numIslands(self, grid: List[List[str]]) -> int:
        if len(grid) == 0:
            return 0

        rowLen = len(grid)
        colLen = len(grid[0])
        self.count = 0

        # initialize a parent array where each element in the 0 indexed matrix
        # is it's own parent
        parent = [i for i in range(rowLen*colLen)]
        rank = [0] * rowLen*colLen

        # initialize minimum island count to the count of all the individual lands in the matrix
        # cause if none of the lands are conncted to each other then we have L islands count total (max count of islands)
        for i in range(rowLen):
            for j in range(colLen):
                if grid[i][j] == '1':
                    self.count += 1

        def find(x):
            if x != parent[x]:
                parent[x] = find(parent[x])

            return parent[x]

        def union(x, y):
            xroot = find(x)
            yroot = find(y)

            if xroot == yroot:
                return
            if rank[xroot] < rank[yroot]:
                xroot, yroot = yroot, xroot

            parent[yroot] = xroot
            rank[xroot] = max(rank[xroot], rank[yroot]+1)

            # If we are joining islands then we are reducing the count of individual islands by 1
            self.count -= 1

        for i in range(rowLen):
            for j in range(colLen):
                if grid[i][j] == '0':
                    continue
                index = i*colLen + j
                if j < colLen-1 and grid[i][j+1] == '1':
                    union(index, index+1)
                if i < rowLen-1 and grid[i+1][j] == '1':
                    union(index, index+colLen)
        return self.count

In [ ]:
# LC Union Find sol
class Solution:
    def numIslands(self, grid: List[List[str]]) -> int:
        if not grid or not grid[0]:
            return 0

        nr = len(grid)
        nc = len(grid[0])
        uf = UnionFind(grid)

        for r in range(nr):
            for c in range(nc):
                if grid[r][c] == "1":
                    grid[r][c] = "0"
                    if r - 1 >= 0 and grid[r - 1][c] == "1":
                        uf.union(r * nc + c, (r - 1) * nc + c)
                    if r + 1 < nr and grid[r + 1][c] == "1":
                        uf.union(r * nc + c, (r + 1) * nc + c)
                    if c - 1 >= 0 and grid[r][c - 1] == "1":
                        uf.union(r * nc + c, r * nc + c - 1)
                    if c + 1 < nc and grid[r][c + 1] == "1":
                        uf.union(r * nc + c, r * nc + c + 1)

        return uf.getCount()

class UnionFind:
    def __init__(self, grid):
        self.count = 0
        m, n = len(grid), len(grid[0])
        self.parent = []
        self.rank = []
        for i in range(m):
            for j in range(n):
                if grid[i][j] == "1":
                    self.parent.append(i * n + j)
                    self.count += 1
                else:
                    self.parent.append(-1)
                self.rank.append(0)

    def find(self, i):
        if self.parent[i] != i:
            self.parent[i] = self.find(self.parent[i])
        return self.parent[i]

    def union(self, x, y):
        rootx = self.find(x)
        rooty = self.find(y)
        if rootx != rooty:
            if self.rank[rootx] > self.rank[rooty]:
                self.parent[rooty] = rootx
            elif self.rank[rootx] < self.rank[rooty]:
                self.parent[rootx] = rooty
            else:
                self.parent[rooty] = rootx
                self.rank[rootx] += 1
            self.count -= 1

    def getCount(self):
        return self.count